In [29]:
!pip install -q wrds
import wrds

!pip install bs4 -q
from bs4 import BeautifulSoup

!pip install  transformers -q
from transformers import pipeline

!pip install sentencepiece -q
import sentencepiece

!pip install openai -q
import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.8 MB/s eta 0:00:00


In [30]:
import os
import re
import pandas as pd
import requests
import pickle
import tensorflow as tf
from PIL import Image
from io import BytesIO
from tqdm import tqdm
headers={'User-Agent': 'email@gmail.com'}

In [31]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [34]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-large-squad2")

model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-large-squad2")

qa_model = pipeline('question-answering', model=model, tokenizer=tokenizer) #Using CPUs. add argument device=0  if running on GPUs.

In [ ]:
#Set up connection to WRDS. Need WRDS username and Password
db = wrds.Connection()

In [ ]:
#Get Def14A (First 10 Def14A of the year 2020)
sql_query ="""
SELECT *
FROM wrdssec_all.forms
WHERE form='DEF 14A' and fdate>'2020-01-01' and fdate<'2020-02-01' and gvkey != 'None'
LIMIT 10
"""
def14 = db.raw_sql(sql_query) 
def14['seclink']='https://www.sec.gov/Archives/'+def14.fname
def14

In [61]:
#Get and clean Def14A
def get_clean_text(url):
  page=requests.get(url, headers=headers).text
  soup = BeautifulSoup(page)
  text=soup.get_text("\n\n")

  text=re.sub(r'\n\n',' ',text)
  text=re.sub(r'\n',' ',text)
  text=re.sub(r'&#160;|&nbsp;|•',' ',text)
  text=re.sub(r'\xa0|\x92|\x93|\x94|\x95|\x96|\x97|\x98|\x99',' ',text)

  text=text.replace('  ','')
  text=text.replace('\.','')
  text=text.replace('>','')
  return text

In [149]:
def get_near_words(url):
  # Define the word you want to search for
  words = ["median"]

  # Split the text into a list of words or numbers
  words_numbers_chars = re.findall(r"[\w']+|[\d.]+|[^\w\s]", get_clean_text(url))

  # Find all instances of the word in the text
  indices = [i for i, w in enumerate(words_numbers_chars) if w.lower() in words]

  # Extract the context for each instance of the word

  output=[]
  for index in indices:
      start = max(0, index - 500)
      end = min(len(words_numbers_chars), index + 500)
      context = words_numbers_chars[start:end]
      temp= " ".join(context)
      output.append(temp)

  return  " ".join(output)

In [150]:
#Setup question and answering pipeline using huggingface model.
def qa(url):
  try:
    answer=qa_model(question = "What is the ratio of ceo salary with median employee salary?",
            context = get_near_words(url))
    return answer['answer']
  except:
    return 'not available'

In [ ]:
#Result using Hugginface models
def14[0:3].seclink.apply(lambda x: qa(x))

In [ ]:
#Using ChatGPT
!pip install openai -q
import openai

In [138]:
#Setup question and answering pipeline using OpenAI model.
def qa_openai(url):
  openai.api_key = "your api keys goes here"

  response = openai.Completion.create(
    model="text-davinci-003",
    prompt=get_near_words(url) + '''
    
    "What is the ratio of ceo salary with median employee salary? If answer cannot be inferred from the text, please say not avaiable."
''',
    temperature=0,
    max_tokens=20,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
  )

  return response.choices[0]["text"]

In [ ]:
#Result using CHATGPT models
def14[0:3].seclink.apply(lambda x: qa_openai(x))